In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras import layers
import keras

In [43]:
def model(input_shape):
    x = layers.Input(shape=(input_shape,))

    embedding = layers.Embedding(5, 4, mask_zero=True)
    flatten = layers.Flatten()
    dense = layers.Dense(units=1, activation='softmax')
    
    e = embedding(x)
    p = flatten(e)
    y = dense(p)
    model = keras.Model(x, y)
    return model

In [44]:
m = model(4)
m.summary()

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 4)]               0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 4, 4)              20        
_________________________________________________________________
flatten_2 (Flatten)          (None, 16)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 37
Trainable params: 37
Non-trainable params: 0
_________________________________________________________________


In [38]:
x = [[1, 4, 3, 1], [1, 2, 2, 1], [4, 1, 1, 0]]
y = [[1], [0], [0]]

In [39]:

optimizer = 'adam'
loss = keras.losses.CategoricalCrossentropy()
metrics = ['AUC']
epochs = 20

m.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [40]:
m.fit(x, y)

1/1 [==============================] - 0s 318ms/step - loss: 0.0000e+00 - auc: 0.5000
